In [ ]:
from progressivis_nb_widgets import *

In [ ]:
from progressivis import Scheduler
from progressivis.cluster import MBKMeans, MBKMeansFilter
from progressivis.io import CSVLoader
from progressivis.vis import MCScatterPlot
from progressivis.core.utils import gather_and_run
from progressivis.utils.psdict import PsDict
from progressivis.stats.blobs_table import BlobsTable, MVBlobsTable
import numpy as np

s = Scheduler()
MULTIVARIATE_NORMAL = True
if not MULTIVARIATE_NORMAL: # simple isotropic blobs (using scikit-learn make_blobs)
    centers = [(0.1, 0.3), (0.7, 0.5), (-0.4, -0.3)]
    data = BlobsTable(columns=['_0', '_1'],  centers=centers, cluster_std=0.2, rows=999999, scheduler=s)
else: # i.e. control each cluster's covariance matrix (using numpy.random.multivariate_normal)
    means = [0.1, 0.3], [0.7, 0.5], [-0.4, -0.3]
    covs = [[0.01, 0], [0, 0.09]], [[0.04, 0], [0, 0.01]], [[0.09, 0.04], [0.04, 0.02]]
    data = MVBlobsTable(columns=['_0', '_1'], means=means, covs=covs, rows=2_000_000, scheduler=s)
n_clusters = 3
mbkmeans = MBKMeans(columns=['_0', '_1'], n_clusters=n_clusters, 
                    batch_size=100, tol=0.01, is_input=False, scheduler=s)
mbkmeans.create_dependent_modules(data)
classes = []
for i in range(n_clusters):
    cname = f"k{i}"
    filt = MBKMeansFilter(i, scheduler=s)
    filt.create_dependent_modules(mbkmeans, data, 'result')
    classes.append({'name': cname, 'x_column': '_0',
                    'y_column': '_1', 'sample': mbkmeans if i==0 else None,
                    'sample_slot': 'result',
                    'input_module': filt, 'input_slot': 'result'})
sp = MCScatterPlot(scheduler=s, classes=classes)
sp.create_dependent_modules()
for i in range(n_clusters):
    cname = f"k{i}"
    sp[cname].min_value.result = PsDict({'_0': -np.inf, '_1': -np.inf})
    sp[cname].max_value.result = PsDict({'_0': np.inf, '_1': np.inf})

sp.move_point = mbkmeans.moved_center # for input management
# Create the dashboard object
psboard = PsBoard(s, order='desc')
# Visualisations require registration :
# 1) When widget provides the link_module() method do :
psboard.register_visualisation(Scatterplot(disable=['filter']), sp)
# 2) When widget is generic or for overloading the link_module() method do:
# psboard.register_visualisation(FooWidget(), bar_module, glue=glue_func)
# NB: glue_func(widget, module)-> [coroutine] must be provided
# Start application :
gather_and_run(s.start(), *psboard.coroutines)

#### Click  _&#x1f441; mc_scatter_plot_1_ below to display the scatterplot:

In [ ]:
display(psboard)